<a href="https://colab.research.google.com/github/BrotherKim/Colab/blob/main/SEP531/Wk5_P1_WebText%2BSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [SEP 531] 정보검색 실습: Text Classification using Crawled Data

Primary TA: 임채균

TA's E-mail: rayote@kaist.ac.kr

본 실습에서는 텍스트 데이터를 대상으로 감성 분석, 문서 분류에 대한 전형적인 모델을 학습하고 성능을 확인하는 방법을 다룹니다. 


## Contents

1. 감성 분석 (네이버 영화리뷰)
  * 네이버 영화리뷰 데이터셋
  * 모델 학습 (Naive Bayse, k-NN, SVM)

2. 문서 분류 (뉴스그룹 데이터)
  * 뉴스그룹 데이터셋 (sklearn 패키지)
  * 모델 학습 (Naive Bayse, k-NN, SVM)

* 실습: 모델 학습성능 개선 (데이터 전처리, 파라미터 튜닝 등)
  * 위 1, 2번 중에서 임의의 데이터셋, 모델 선정
  * 예시 성능보다 향상된 학습결과 달성


# Part 1: 감성 분석 (네이버 영화리뷰)


## (1) 네이버 영화리뷰 데이터셋

#### **Step 01: 데이터 다운로드**

In [ ]:
!wget -O ratings_train.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -O ratings_test.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2021-10-05 12:24:12--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.07s   

2021-10-05 12:24:13 (197 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2021-10-05 12:24:13--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving

#### **Step 02: 학습/테스트 데이터셋 로드**

**학습데이터 "ratings_train.txt"**
* 한글 처리를 위해 UTF-8 인코딩
* 3개 컬럼으로 구성: id, document, label
  * document: 리뷰 텍스트
  * label: 감성 클래스, negative (label=0), positive (label=1)

In [ ]:
with open('ratings_train.txt', encoding='utf-8') as f:
  raw_data = f.read()
  print(raw_data[:200])

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서


In [ ]:
# 문장 단위, 탭 문자 기준으로 데이터 분리
data = []
for line in raw_data.splitlines():
  data.append(line.split('\t'))

# 헤더 부분 삭제
data = data[1:]
print(len(data))

# 데이터 10,000개만 발췌
data = data[:10000]

print(data[:3])

150000
[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0']]


In [ ]:
import numpy as np

# 학습 데이터셋 구성
# data 객체를 zip함수로 묶었다가 1번(document) 컬럼만 선택
x_train = list(zip(*data))[1]
print(x_train[:3])

# data 객체를 zip함수로 묶었다가 2번(label) 컬럼만 선택
y_train = list(zip(*data))[2]
# numpy 정수 배열로 변환
y_train = np.array(y_train, dtype=int)
print(y_train[:3])

('아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다')
[0 1 0]


**테스트데이터 "ratings_test.txt"**
* 위 학습셋과 동일한 과정으로 처리


In [ ]:
# 위와 같은 과정으로 테스트 데이터 처리
with open('ratings_test.txt', encoding='utf-8') as f:
  raw_data = f.read()

# 문장 단위, 탭 문자 기준으로 데이터 분리
data = []
for line in raw_data.splitlines():
  data.append(line.split('\t'))

# 헤더 부분 삭제
data = data[1:]
print(len(data))

# 데이터 2,500개만 발췌
data = data[:2500]

# 테스트 데이터셋 구성
x_test = list(zip(*data))[1]
print(x_test[:3])

y_test = list(zip(*data))[2]
y_test = np.array(y_test, dtype=int)
print(y_test[:3])

50000
('굳 ㅋ', 'GDNTOPCLASSINTHECLUB', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아')
[1 0 0]


## (2) 모델 학습 수행

#### **Step 01: NaiveBayes 모델 학습**

![Naive Bayes](https://miro.medium.com/max/2400/1*39U1Ln3tSdFqsfQy6ndxOA.png)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model_nb = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [ ]:
%%time
model_nb.fit(x_train, y_train)

CPU times: user 194 ms, sys: 5.89 ms, total: 200 ms
Wall time: 210 ms


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
# 테스트셋으로 성능 측정
result_review_nb = classification_report(y_test, model_nb.predict(x_test))
print(result_review_nb)

              precision    recall  f1-score   support

           0       0.75      0.80      0.78      1224
           1       0.80      0.74      0.77      1276

    accuracy                           0.77      2500
   macro avg       0.77      0.77      0.77      2500
weighted avg       0.77      0.77      0.77      2500



#### **Step 02: k-NN 모델 학습**

![k-NN](https://wikidocs.net/images/page/32057/%EA%B7%B8%EB%A6%BC12.png)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = Pipeline([
    ('vect', TfidfVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
])

In [ ]:
%%time
model_knn.fit(x_train, y_train)

CPU times: user 141 ms, sys: 8.3 ms, total: 149 ms
Wall time: 148 ms


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [ ]:
# 테스트셋으로 성능 측정
result_review_knn = classification_report(y_test, model_knn.predict(x_test))
print(result_review_knn)

              precision    recall  f1-score   support

           0       0.54      0.95      0.69      1224
           1       0.82      0.21      0.34      1276

    accuracy                           0.58      2500
   macro avg       0.68      0.58      0.51      2500
weighted avg       0.68      0.58      0.51      2500



#### **Step 03: SVM 모델 학습**

<!-- ![Support Vector](https://miro.medium.com/max/1400/1*TSjvQlDde90FQtoD9-9iHw.png) -->

![SVM Kernerls](https://miro.medium.com/max/1400/1*qZFPN60NvwbFE_tRg2YPoQ.png)


**SVM(linear) 모델 학습**


In [ ]:
from sklearn.svm import SVC
model_svm = Pipeline([
    ('vect', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1e10)),
])


In [ ]:
%%time
model_svm.fit(x_train, y_train)

CPU times: user 12.1 s, sys: 48.6 ms, total: 12.1 s
Wall time: 12.1 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 SVC(C=10000000000.0, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     deci

In [ ]:
# 테스트셋으로 성능 측정
result_review_svm = classification_report(y_test, model_svm.predict(x_test))
print(result_review_svm)

              precision    recall  f1-score   support

           0       0.75      0.63      0.69      1224
           1       0.70      0.80      0.74      1276

    accuracy                           0.72      2500
   macro avg       0.72      0.72      0.72      2500
weighted avg       0.72      0.72      0.72      2500



**SVM(sigmoid) 모델 학습**

In [ ]:
model_svm_sigmoid = Pipeline([
    ('vect', TfidfVectorizer()),
    ('svm', SVC(kernel='sigmoid', gamma=3, coef0=1)),
])


In [ ]:
%%time
model_svm_sigmoid.fit(x_train, y_train)

CPU times: user 8.18 s, sys: 31.9 ms, total: 8.22 s
Wall time: 8.18 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree

In [ ]:
# 테스트셋으로 성능 측정
result_review_svm_sig = classification_report(y_test, model_svm_sigmoid.predict(x_test))
print(result_review_svm_sig)

              precision    recall  f1-score   support

           0       0.70      0.83      0.76      1224
           1       0.80      0.66      0.72      1276

    accuracy                           0.74      2500
   macro avg       0.75      0.74      0.74      2500
weighted avg       0.75      0.74      0.74      2500



#### *전체 학습모델 성능비교 (네이버 영화리뷰)*

In [ ]:
# 네이버 영화리뷰에 대한 전체 학습모델 성능비교
def print_acc(name, str_report):
  start = str_report.index('accuracy')
  end = str_report.index('\n', start)
  print('< %s >' % name)
  print('    ' + str_report[start:end])

print_acc('Naive Bayes', result_review_nb)
print_acc('k-NN', result_review_knn)
print_acc('SVM (linear)', result_review_svm)
print_acc('SVM (sigmoid)', result_review_svm_sig)

< Naive Bayes >
    accuracy                           0.77      2500
< k-NN >
    accuracy                           0.58      2500
< SVM (linear) >
    accuracy                           0.72      2500
< SVM (sigmoid) >
    accuracy                           0.74      2500


# Part 2: 문서 분류 (뉴스그룹 데이터)


## (1) 뉴스그룹 데이터셋

* 타겟 데이터
  * 문서가 속한 뉴스 그룹 (총 20개 그룹)
  
* 학습 데이터
  * 문서 텍스트

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset='all')
print(newsgroups.DESCR)

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality 

In [ ]:
# 타겟 그룹 20개
from pprint import pprint
pprint(list(newsgroups.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [ ]:
# 학습 데이터 예시 (1번 샘플)
print(newsgroups.data[1])

From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)
Subject: Which high-performance VLB video card?
Summary: Seek recommendations for VLB video card
Nntp-Posting-Host: midway.ecn.uoknor.edu
Organization: Engineering Computer Network, University of Oklahoma, Norman, OK, USA
Keywords: orchid, stealth, vlb
Lines: 21

  My brother is in the market for a high-performance video card that supports
VESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:

  - Diamond Stealth Pro Local Bus

  - Orchid Farenheit 1280

  - ATI Graphics Ultra Pro

  - Any other high-performance VLB card


Please post or email.  Thank you!

  - Matt

-- 
    |  Matthew B. Lawson <------------> (mblawson@essex.ecn.uoknor.edu)  |   
  --+-- "Now I, Nebuchadnezzar, praise and exalt and glorify the King  --+-- 
    |   of heaven, because everything he does is right and all his ways  |   
    |   are just." - Nebuchadnezzar, king of Babylon, 562 B.C.           |   



In [ ]:
# 타겟 데이터 예시 (1번 샘플)
print(newsgroups.target_names[newsgroups.target[1]])

comp.sys.ibm.pc.hardware


## (2) 모델 학습 수행

#### **Step 00: 학습/테스트 데이터셋**

In [ ]:
print(newsgroups.data[1])
print(newsgroups.target[1])

From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)
Subject: Which high-performance VLB video card?
Summary: Seek recommendations for VLB video card
Nntp-Posting-Host: midway.ecn.uoknor.edu
Organization: Engineering Computer Network, University of Oklahoma, Norman, OK, USA
Keywords: orchid, stealth, vlb
Lines: 21

  My brother is in the market for a high-performance video card that supports
VESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:

  - Diamond Stealth Pro Local Bus

  - Orchid Farenheit 1280

  - ATI Graphics Ultra Pro

  - Any other high-performance VLB card


Please post or email.  Thank you!

  - Matt

-- 
    |  Matthew B. Lawson <------------> (mblawson@essex.ecn.uoknor.edu)  |   
  --+-- "Now I, Nebuchadnezzar, praise and exalt and glorify the King  --+-- 
    |   of heaven, because everything he does is right and all his ways  |   
    |   are just." - Nebuchadnezzar, king of Babylon, 562 B.C.           |   

3


3

In [ ]:
from sklearn.model_selection import train_test_split

# 전체 데이터를 셔플 후, 학습/테스트셋으로 분리
x_train, x_test, y_train, y_test = train_test_split(newsgroups.data, 
                                                    newsgroups.target, 
                                                    shuffle=True,
                                                    random_state=15)

# 학습셋 4,000개만 선택
NUM_TRAIN = 4000
x_train = x_train[:NUM_TRAIN]
y_train = y_train[:NUM_TRAIN]

# 테스트셋 2,500개만 선택
NUM_TEST = 2500
x_test = x_test[:NUM_TEST]
y_test = y_test[:NUM_TEST]

#### **Step 01: NaiveBayes 모델 학습**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

text_model_nb = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [ ]:
%%time
text_model_nb.fit(x_train, y_train)

CPU times: user 915 ms, sys: 11.4 ms, total: 926 ms
Wall time: 922 ms


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
# 테스트셋으로 성능 측정
result_news_nb = classification_report(y_test, text_model_nb.predict(x_test), target_names=newsgroups.target_names)
print(result_news_nb)

                          precision    recall  f1-score   support

             alt.atheism       0.93      0.79      0.86       102
           comp.graphics       0.91      0.29      0.44       137
 comp.os.ms-windows.misc       0.83      0.74      0.78       129
comp.sys.ibm.pc.hardware       0.74      0.73      0.73       144
   comp.sys.mac.hardware       0.78      0.75      0.76       140
          comp.windows.x       0.85      0.75      0.80       142
            misc.forsale       0.87      0.77      0.82       131
               rec.autos       0.77      0.83      0.80       120
         rec.motorcycles       0.88      0.91      0.89       128
      rec.sport.baseball       0.95      0.93      0.94       132
        rec.sport.hockey       0.94      0.94      0.94       131
               sci.crypt       0.65      0.92      0.76       123
         sci.electronics       0.81      0.58      0.67       139
                 sci.med       0.76      0.89      0.82       114
         

#### **Step 02: k-NN 모델 학습**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
text_model_knn = Pipeline([
    ('vect', TfidfVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
])

In [ ]:
%%time
text_model_knn.fit(x_train, y_train)

CPU times: user 844 ms, sys: 9.04 ms, total: 853 ms
Wall time: 851 ms


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,


In [ ]:
# 테스트셋으로 성능 측정
result_news_knn = classification_report(y_test, text_model_knn.predict(x_test), target_names=newsgroups.target_names)
print(result_news_knn)

                          precision    recall  f1-score   support

             alt.atheism       0.42      0.84      0.56       102
           comp.graphics       0.55      0.50      0.52       137
 comp.os.ms-windows.misc       0.59      0.67      0.63       129
comp.sys.ibm.pc.hardware       0.56      0.60      0.58       144
   comp.sys.mac.hardware       0.56      0.47      0.51       140
          comp.windows.x       0.69      0.61      0.65       142
            misc.forsale       0.59      0.44      0.50       131
               rec.autos       0.68      0.62      0.65       120
         rec.motorcycles       0.83      0.75      0.79       128
      rec.sport.baseball       0.82      0.69      0.75       132
        rec.sport.hockey       0.82      0.76      0.79       131
               sci.crypt       0.81      0.75      0.78       123
         sci.electronics       0.71      0.48      0.58       139
                 sci.med       0.84      0.67      0.74       114
         

#### **Step 03: SVM 모델 학습**

**SVM(linear) 모델 학습**

In [ ]:
from sklearn.svm import SVC
text_model_svm = Pipeline([
    ('vect', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1e10)),
])


In [ ]:
%%time
text_model_svm.fit(x_train, y_train)

CPU times: user 34.3 s, sys: 22.1 ms, total: 34.3 s
Wall time: 34.2 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 SVC(C=10000000000.0, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     deci

In [ ]:
# 테스트셋으로 성능 측정
result_news_svm = classification_report(y_test, text_model_svm.predict(x_test), target_names=newsgroups.target_names)
print(result_news_svm)

                          precision    recall  f1-score   support

             alt.atheism       0.89      0.87      0.88       102
           comp.graphics       0.75      0.77      0.76       137
 comp.os.ms-windows.misc       0.80      0.81      0.81       129
comp.sys.ibm.pc.hardware       0.74      0.76      0.75       144
   comp.sys.mac.hardware       0.82      0.81      0.82       140
          comp.windows.x       0.81      0.78      0.80       142
            misc.forsale       0.77      0.86      0.81       131
               rec.autos       0.88      0.91      0.89       120
         rec.motorcycles       0.97      0.92      0.94       128
      rec.sport.baseball       0.93      0.95      0.94       132
        rec.sport.hockey       0.98      0.91      0.94       131
               sci.crypt       0.95      0.85      0.90       123
         sci.electronics       0.66      0.85      0.74       139
                 sci.med       0.85      0.90      0.88       114
         

**SVM(sigmoid) 모델 학습**

In [ ]:
text_model_svm_sigmoid = Pipeline([
    ('vect', TfidfVectorizer()),
    ('svm', SVC(kernel='sigmoid', gamma=4, coef0=1)),
])


In [ ]:
%%time
text_model_svm_sigmoid.fit(x_train, y_train)

CPU times: user 22.3 s, sys: 16.8 ms, total: 22.3 s
Wall time: 22.2 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svm',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree

In [ ]:
# 테스트셋으로 성능 측정
result_news_svm_sig = classification_report(y_test, text_model_svm_sigmoid.predict(x_test), target_names=newsgroups.target_names)
print(result_news_svm_sig)

                          precision    recall  f1-score   support

             alt.atheism       0.78      0.80      0.79       102
           comp.graphics       0.73      0.64      0.68       137
 comp.os.ms-windows.misc       0.78      0.75      0.76       129
comp.sys.ibm.pc.hardware       0.66      0.70      0.68       144
   comp.sys.mac.hardware       0.82      0.74      0.78       140
          comp.windows.x       0.78      0.76      0.77       142
            misc.forsale       0.80      0.86      0.83       131
               rec.autos       0.79      0.87      0.83       120
         rec.motorcycles       0.95      0.85      0.90       128
      rec.sport.baseball       0.92      0.92      0.92       132
        rec.sport.hockey       0.96      0.91      0.93       131
               sci.crypt       0.94      0.84      0.89       123
         sci.electronics       0.52      0.82      0.64       139
                 sci.med       0.83      0.89      0.86       114
         

#### *전체 학습모델 성능비교 (뉴스그룹)*

In [ ]:
# 뉴스그룹 데이터셋에 대한 전체 학습모델 성능비교
def print_acc(name, str_report):
  start = str_report.index('accuracy')
  end = str_report.index('\n', start)
  print('< %s >' % name)
  print('    ' + str_report[start:end])

print_acc('Naive Bayes', result_news_nb)
print_acc('k-NN', result_news_knn)
print_acc('SVM (linear)', result_news_svm)
print_acc('SVM (sigmoid)', result_news_svm_sig)

< Naive Bayes >
    accuracy                           0.76      2500
< k-NN >
    accuracy                           0.65      2500
< SVM (linear) >
    accuracy                           0.85      2500
< SVM (sigmoid) >
    accuracy                           0.80      2500


# 실습: 모델 학습성능 개선
  * **수행방법**:
    - 상기 실습내용 중에서 임의의 모델 선정
    - 아래의 크롤러 샘플 코드를 활용하여, "네이버 뉴스"의 데이터를 수집
    - 데이터 전처리, 파라미터 튜닝 등 자유로운 방법을 통해 성능 개선을 시도
  * **달성 목표:** 
    - "네이버 뉴스"에서 <u>최소 3개 이상</u>의 도메인 확보 (e.g., 정치, 경제, 생활/문화, IT/과학 등)
    - 실행결과에서 확인할 수 있는 <u>예시 성능보다 향상</u>된 학습결과 달성

In [93]:
##########
# TODO: 학습/데스트 데이터셋 구성
##########

import numpy as np

# 학습 데이터셋 구성
from sklearn.model_selection import train_test_split

itData = crawl_naver_news('it')
economyData = crawl_naver_news('economy')
worldData = crawl_naver_news('world')

itLabel = [105 for i in range(len(itData))]
economyLabel = [101 for i in range(len(economyData))]
worldLabel = [104 for i in range(len(worldData))]

allData = np.concatenate((itData, economyData, worldData), axis=None)
allLabel = np.concatenate((itLabel, economyLabel, worldLabel), axis=None)

# 전체 데이터를 셔플 후, 학습/테스트셋으로 분리
x_train, x_test, y_train, y_test = train_test_split(allData, 
                                                    allLabel, 
                                                    shuffle=True,
                                                    random_state=15)

# 학습셋 4,000개만 선택
#NUM_TRAIN = 145
#x_train = x_train[:NUM_TRAIN]
#y_train = y_train[:NUM_TRAIN]

# 테스트셋 2,500개만 선택
#NUM_TEST = 2500
#x_test = x_test[:NUM_TEST]
#y_test = y_test[:NUM_TEST]


##########
# TODO: 모델 구성
##########
model = Pipeline([
    ('vect', TfidfVectorizer()),
    ('svm', SVC(kernel='sigmoid', gamma=3, coef0=1)),
])


# 모델 학습
model.fit(x_train, y_train)

# 테스트셋으로 성능 측정
print( classification_report(y_test, model.predict(x_test)) )

Parsing a page 1 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 2 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 3 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 4 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 5 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 6 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 7 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 8 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 9 (of total 10)..
Waiting a moment before next crawling..
Parsing a page 10 (of total 10)..
Waiting a moment before next crawling..
Parsing a news from "https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&oid=421&aid=0005638842"..
공정위원장 "카카오 금산분리 위반 조사 중…신속 마무리 노력"
Waiting a moment before next crawling..
Parsing a news from "https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid

**(참고) 네이버 뉴스 크롤러 구현 코드**


In [92]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import copy
import time

# 네이버 뉴스 도메인별 id
NAVER_NEWS_DOMAIN = {
    'economy': '101', 
    'world': '104',
    'it': '105',
}
# 다음 크롤링 대기시간
#DELAY_WAIT_SEC = 3
DELAY_WAIT_SEC = 1

# 크롤링 페이지 개수
NUM_PAGES = 10

#######################################
# 대상 URL의 페이지 로드
#######################################
def request_headless(url):
  req = Request(url)
  req.add_header('user-agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36')
  client = urlopen(req)
  page = client.read()
  client.close()

  return page

#######################################
# 뉴스 목록 파싱
#######################################
def parse_list_news(url):
  temp_html = request_headless(url)
  bs = BeautifulSoup(temp_html, 'html.parser')
  # 헤드라인 클러스터 메인 뉴스
  list_news_a = bs.select('a.cluster_thumb_link')
  # 헤드라인 클러스터 목록별 뉴스
  list_news_b = bs.select('div.cluster_text a')
  # 뉴스 URL 부분만 추출
  parse_next = []
  for tag in list_news_a:
    parse_next.append(tag['href'])
  for tag in list_news_b:
    parse_next.append(tag['href'])

  return parse_next

#######################################
# 목표 페이지 개수만큼 뉴스 목록 파싱
#######################################
def get_parse_next(url, num_page):
  list_news_all = []
  for p in range(num_page):
    page_next = p + 1
    print('Parsing a page ' + str(page_next) + ' (of total ' + str(num_page) + ')..')
    target_url = url +'&page=' + str(page_next)
    temp_list_next = parse_list_news(target_url)
    list_news_all = list_news_all + temp_list_next
    print('Waiting a moment before next crawling..')
    time.sleep(DELAY_WAIT_SEC)    # 다음 작업수행 전 대기

  return list_news_all

#######################################
# 뉴스 본문 파싱
#######################################
def parse_news_content(url):
  page = request_headless(url)

  # HTML 파서 로드
  bs = BeautifulSoup(page, 'html.parser')
  # 제목 추출
  title = bs.select_one('h3#articleTitle').text
  print(title)
  # 본문 클래스를 포함한 단일 태그를 select
  body = bs.select_one('div._article_body_contents')
  # 대상 태그를 복제
  tempBody = copy.copy(body)
  # decompose 함수를 적용하여 "script" 파트를 삭제
  tempBody.script.decompose()
  # 결과 출력
  content = tempBody.text.strip()
  
  #obj = {'doc_title': title, 'news_content': content}
  obj = 'doc_title:{0}\nnews_content:{1}'.format(title, content)
  # pprint(obj)

  return obj

#######################################
# 네이버 뉴스 크롤링
#######################################
def crawl_naver_news(target_domain):
  # 뉴스 페이지 목록 요청
  url = 'https://news.naver.com/main/main.naver?mode=LS2D&mid=shm&sid1=' + NAVER_NEWS_DOMAIN[target_domain]
  list_news_all = get_parse_next(url, NUM_PAGES)

  # 뉴스 본문 파싱 후 결과목록 저장
  results = []
  for url_news in list_news_all:
    print('Parsing a news from "' + url_news + '"..')
    results.append(parse_news_content(url_news))
    print('Waiting a moment before next crawling..')
    time.sleep(DELAY_WAIT_SEC)    # 다음 작업수행 전 대기

  return results


#results = crawl_naver_news('it')

In [ ]:
len(results)

41

In [ ]:
print(results[1])

{'doc_title': '데이터 활용 늘고 신산업서 성과…디지털 전환 ‘착착’', 'news_content': '과기정통부, ‘2021 4차 산업혁명 지표’ 발표데이터·네트워크·인공지능(D·N·A) 인프라·산업 성장세금융 핀테크 확산세, 디지털정부 지수 등 눈길2021년 4차 산업혁명 주표 지표 (자료=과기정통부)[이데일리 이대호 기자] 과학기술정보통신부(장관 임혜숙, 과기정통부)는 4차 산업혁명의 기반인 데이터·네트워크·인공지능(D.N.A.) 성장과 혁신의 성과를 보여주는 ’2021 4차 산업혁명 지표‘를 5일 발표했다.과기정통부는 4차 산업혁명 대응계획에 따라 2019년부터 국민이 체감할 수 있는 성과지표를 발굴·선정해 발표하고 있다. 이번 지표는 지난 2년간 발표한 지표를 기초로 그간 데이터·네트워크·인공지능(D·N·A) 정책성과 및 ’디지털 뉴딜‘ 등 최근 정책방향을 국민들이 쉽게 체감할 수 있도록 선정·조사했다는 설명이다.주요 지표에 따르면 디지털 전환의 핵심 인프라인 데이터·네트워크·인공지능(D·N·A) 분야의 확장이 지속되고 있다.데이터 산업의 전체 시장규모는 19조2736억원(작년기준 추정)으로 전년 대비 14.3% 성장했다. 공공데이터 개방 건수는 5만5561건(올해 3월)으로 전년 대비 63.4% 증가해 데이터경제 활성화를 위한 기반을 제공 중이다.네트워크 분야인 사물인터넷(IoT) 서비스 가입수는 3098만개로(올해 6월) 전년 대비 18.8% 증가했다. 인터넷동영상서비스(OTT) 이용률은 2019년 52.0%에서 2020년 66.3%로 14.3% 늘었다.인공지능(AI) 산업 매출액은 6895억원(작년기준 추정)으로 전년 대비 16.3% 성장했고 고도화된 인공지능 개발을 위한 AI 학습용 데이터 활용 횟수도 전년대비 153.4% 늘어난 9만6826회(올해 8월 누적 기준)를 기록했다.가상·증강현실(VR·AR) 산업 매출액은 작년 8032억원(전년대비 6.8%↑)을 기록했고 참여 종사자(2019년 5940명, 전년대비 10.6%↑) 모두 성